### Train Dense model

In [ ]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

ROOT_DIR = os.path.abspath("../")

### Load data

In [ ]:
# Load data
data = np.load(os.path.join(ROOT_DIR, "datasets", "train_1D.npz"))
x_train = data['name1']
y_train = data['name2']
sym_train = data['name3']

data = np.load(os.path.join(ROOT_DIR, "datasets", "val_1D.npz"))
x_val = data['name1']
y_val = data['name2']
sym_val = data['name3']

data = np.load(os.path.join(ROOT_DIR, "datasets", "test_1D.npz"))
x_test = data['name1']
y_test = data['name2']
sym_test = data['name3']

print(x_train.shape, y_train.shape, len(sym_train))
print(x_val.shape, y_val.shape, len(sym_val))
print(x_test.shape, y_test.shape, len(sym_test))

### Build model

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.utils import to_categorical

model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = x_train.shape[1]))
model.add(Dropout(rate = 0.25))
model.add(Dense(1, activation = 'sigmoid'))

# compile the model - use categorical crossentropy, and the adam optimizer
model.compile(loss='binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

model.summary()

In [ ]:
# Define callbacks.
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

model.fit(x_train, y_train,
            validation_data=(x_val, y_val),
            batch_size = 512, 
            epochs= 1000, 
            verbose = 1,
            callbacks=[early_stopping_cb])

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score


def calc_prevalence(y_actual):
    return (sum(y_actual)/len(y_actual))


def calc_specificity(y_actual, y_pred, thresh):
    # calculates specificity
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)


def print_report(y_actual, y_pred, thresh):
    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > thresh))
    recall = recall_score(y_actual, (y_pred > thresh))
    precision = precision_score(y_actual, (y_pred > thresh))
    specificity = calc_specificity(y_actual, y_pred, thresh)
    print('AUC:%.3f'%auc)
    print('accuracy:%.3f'%accuracy)
    print('recall:%.3f'%recall)
    print('precision:%.3f'%precision)
    print('specificity:%.3f'%specificity)
    print('prevalence:%.3f'%calc_prevalence(y_actual))
    print(' ')
    return auc, accuracy, recall, precision, specificity


y_train_preds_dense = model.predict_proba(x_train,verbose = 1)
y_valid_preds_dense = model.predict_proba(x_val,verbose = 1)
y_test_preds_dense = model.predict_proba(x_test,verbose = 1)

thresh = (sum(y_train)/len(y_train))[0]
print(thresh)

print('Train-----------')
print_report(y_train, y_train_preds_dense, thresh)
print('Valid-----------')
print_report(y_val, y_valid_preds_dense, thresh)
print('Test-----------')
print_report(y_test, y_test_preds_dense, thresh)

In [ ]:
from keras.layers import Conv1D

model = Sequential()
model.add(Conv1D(filters = 256, kernel_size = 5, activation = 'relu', input_shape = (2160,1)))
model.add(Dropout(rate = 0.25))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))

# compile the model - use categorical crossentropy, and the adam optimizer
model.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

model.summary()

In [ ]:
# reshape input
x_train_cnn = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_val_cnn = np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))
x_test_cnn = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

print(x_train_cnn.shape)
print(x_val_cnn.shape)
print(x_test_cnn.shape)

In [ ]:
# Define callbacks.
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

model.fit(x_train_cnn, y_train,
            validation_data=(x_val_cnn, y_val),
            batch_size = 512, 
            epochs= 1000, 
            verbose = 1,
            callbacks=[early_stopping_cb])

In [ ]:
y_train_preds_cnn = model.predict_proba(x_train_cnn,verbose = 1)
y_valid_preds_cnn = model.predict_proba(x_val_cnn,verbose = 1)
y_test_preds_cnn = model.predict_proba(x_test_cnn,verbose = 1)

print('Train-----------')
print_report(y_train, y_train_preds_cnn, thresh)
print('Valid-----------')
print_report(y_val, y_valid_preds_cnn, thresh)
print('Test-----------')
print_report(y_test, y_test_preds_cnn, thresh)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

fpr_valid_cnn, tpr_valid_cnn, t_valid_cnn = roc_curve(y_val, y_valid_preds_cnn)
auc_valid_cnn = roc_auc_score(y_val, y_valid_preds_cnn)

fpr_valid_dense, tpr_valid_dense, t_valid_dense = roc_curve(y_val, y_valid_preds_dense)
auc_valid_dense = roc_auc_score(y_val, y_valid_preds_dense)

plt.plot(fpr_valid_cnn, tpr_valid_cnn, 'g-', label = 'CNN AUC:%.3f'%auc_valid_cnn)
plt.plot(fpr_valid_dense, tpr_valid_dense, 'r-', label = 'Dense AUC:%.3f'%auc_valid_dense)

plt.plot([0,1],[0,1], 'k--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend(bbox_to_anchor = (1.04,1), loc = 'upper left')
plt.title('Validation Set')
plt.show()

https://www.kaggle.com/gregoiredc/arrhythmia-on-ecg-classification-using-cnn

In [ ]:
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers.normalization import BatchNormalization
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint


def network():
    im_shape=(x_train_cnn.shape[1],1)
    inputs_cnn=Input(shape=(im_shape), name='inputs_cnn')
    conv1_1=Convolution1D(64, (6), activation='relu', input_shape=im_shape)(inputs_cnn)
    conv1_1=BatchNormalization()(conv1_1)
    pool1=MaxPool1D(pool_size=(3), strides=(2), padding="same")(conv1_1)
    conv2_1=Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool1)
    conv2_1=BatchNormalization()(conv2_1)
    pool2=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv2_1)
    conv3_1=Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool2)
    conv3_1=BatchNormalization()(conv3_1)
    pool3=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv3_1)
    flatten=Flatten()(pool3)
    dense_end1 = Dense(64, activation='relu')(flatten)
    dense_end2 = Dense(32, activation='relu')(dense_end1)
    main_output = Dense(1, activation='sigmoid', name='main_output')(dense_end2)
    
    model = Model(inputs= inputs_cnn, outputs=main_output)
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics = ['accuracy'])
    
    return model


model = network()
model.summary()

In [ ]:
# Define callbacks.
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

model.fit(x_train_cnn, y_train,
            validation_data=(x_val_cnn, y_val),
            batch_size = 512, 
            epochs= 1000, 
            verbose = 1,
            callbacks=[early_stopping_cb])

In [ ]:
y_train_preds_cnn = model.predict(x_train_cnn,verbose = 1)
y_valid_preds_cnn = model.predict(x_val_cnn,verbose = 1)
y_test_preds_cnn = model.predict(x_test_cnn,verbose = 1)

print('Train-----------')
print_report(y_train, y_train_preds_cnn, thresh)
print('Valid-----------')
print_report(y_val, y_valid_preds_cnn, thresh)
print('Test-----------')
print_report(y_test, y_test_preds_cnn, thresh)

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Add, Flatten, Activation
from keras import backend as K
from keras.optimizers import Adam


def network2():
    """Implementation of https://arxiv.org/pdf/1805.00794.pdf"""
    
    n_obs, feature, depth = x_train_cnn.shape
    
    inp = Input(shape=(feature, depth))
    C = Conv1D(filters=32, kernel_size=5, strides=1)(inp)

    C11 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(C)
    A11 = Activation("relu")(C11)
    C12 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A11)
    S11 = Add()([C12, C])
    A12 = Activation("relu")(S11)
    M11 = MaxPooling1D(pool_size=5, strides=2)(A12)


    C21 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(M11)
    A21 = Activation("relu")(C21)
    C22 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A21)
    S21 = Add()([C22, M11])
    A22 = Activation("relu")(S11)
    M21 = MaxPooling1D(pool_size=5, strides=2)(A22)


    C31 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(M21)
    A31 = Activation("relu")(C31)
    C32 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A31)
    S31 = Add()([C32, M21])
    A32 = Activation("relu")(S31)
    M31 = MaxPooling1D(pool_size=5, strides=2)(A32)


    C41 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(M31)
    A41 = Activation("relu")(C41)
    C42 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A41)
    S41 = Add()([C42, M31])
    A42 = Activation("relu")(S41)
    M41 = MaxPooling1D(pool_size=5, strides=2)(A42)


    C51 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(M41)
    A51 = Activation("relu")(C51)
    C52 = Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A51)
    S51 = Add()([C52, M41])
    A52 = Activation("relu")(S51)
    M51 = MaxPooling1D(pool_size=5, strides=2)(A52)

    F1 = Flatten()(M51)

    D1 = Dense(32)(F1)
    A6 = Activation("relu")(D1)
    D2 = Dense(32)(A6)
    D3 = Dense(1, activation='sigmoid')(D2)

    model = Model(inputs=inp, outputs=D3)
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics = ['accuracy'])
    
    return model

model = network2()
model.summary()

In [ ]:
# Define callbacks.
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

model.fit(x_train_cnn, y_train,
            validation_data=(x_val_cnn, y_val),
            batch_size = 512, 
            epochs= 1000, 
            verbose = 1,
            callbacks=[early_stopping_cb])

In [ ]:
y_train_preds_cnn = model.predict(x_train_cnn,verbose = 1)
y_valid_preds_cnn = model.predict(x_val_cnn,verbose = 1)
y_test_preds_cnn = model.predict(x_test_cnn,verbose = 1)

print('Train-----------')
print_report(y_train, y_train_preds_cnn, thresh)
print('Valid-----------')
print_report(y_val, y_valid_preds_cnn, thresh)
print('Test-----------')
print_report(y_test, y_test_preds_cnn, thresh)